In [1]:
import re
import random

In [2]:
with open('certcards2.txt', mode='r', encoding='utf8') as f:
    all_cards = f.read()
    
card_split_pattern = r'\n\n\n\d+\n'
all_cards = re.split(card_split_pattern, all_cards)
# Use re.DOTALL to allow . to match newline characters
card_pattern = re.compile(r'(.+?)\n([\s\S]+)', re.DOTALL)
cards = [(match.group(1), match.group(2)) for cstring in all_cards if (match := re.search(card_pattern, cstring))]

# removing the cards that have no content and trimming
cards = [(subject, stripped_content) for subject, content in cards if len(stripped_content := content.strip()) > 5]


# Completely Random Shuffle

In [3]:
random.shuffle(cards)

with open('certcards2.txt', mode='w', encoding='utf8') as f:
    i = 1
    for subject, content in cards:
        f.write('\n'*5)
        f.write(str(i)+'\n')
        f.write(subject+'\n')
        f.write(content)
        i+=1

print(f"{i-1} cards found")

359 cards found
